# Where to open a Restaurant in New York ?

In [1]:
# Import libraries
import pandas as pd
import numpy as np

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported successfully!!')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [2]:
# Get New York Data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
#Create DataFrame
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
neighborhoods['Borough'].unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

#### There are 5 Boroughs in New York !!! Let's explore them.

In [7]:
neighborhoods.shape

(306, 4)

### There are 306 Neighborhoods !!! Let's Visualize them on a map.

In [8]:
# Defining location
address = 'New York, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of New York are {}, {}.'.format(latitude, longitude))

The geographical coordinates of New York are 40.7127281, -74.0060152.


In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [10]:
# Get Bronx Data
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
# Get the other 4 Borough's Data
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
statenisland_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)

## Now, let's get the venues nearby the boroughs

In [12]:
CLIENT_ID = 'HKB1ULVBQPKLEH30AUOZ2V2I4YXF5EDOL1WIWT0EMECCQ1CW' # your Foursquare ID
CLIENT_SECRET = 'FFIUIMUUBC42AIDQ5UG4OYH54234PDCT4OANAHB1SNCJE0HB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [13]:
def getNearbyVenues(names, latitudes, longitudes, boroughs, radius=500):
    
    venues_list=[]
    for name, lat, lng, borough in zip(names, latitudes, longitudes, boroughs):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],                
            v['venue']['categories'][0]['name']) for v in results])       

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                             'Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude',                             
                             'Venue Category']
    
    return(nearby_venues)

In [14]:
# Get nearby venues in Bronx
boroughs=bronx_data['Borough']
names=bronx_data['Neighborhood']
latitudes=bronx_data['Latitude']
longitudes=bronx_data['Longitude']

radius = 500
LIMIT = 150

bronx_venues = getNearbyVenues(names,latitudes,longitudes,boroughs)

bronx_venues.shape

(1245, 8)

In [15]:
bronx_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Bronx,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [16]:
# Get nearby venues in Manhattan
boroughs=manhattan_data['Borough'] 
names=manhattan_data['Neighborhood']
latitudes=manhattan_data['Latitude']
longitudes=manhattan_data['Longitude']

radius = 500
LIMIT = 150

manhattan_venues = getNearbyVenues(names,latitudes,longitudes,boroughs)

manhattan_venues.shape

(3316, 8)

In [17]:
# Get nearby venues in Brooklyn
boroughs=brooklyn_data['Borough']
names=brooklyn_data['Neighborhood']
latitudes=brooklyn_data['Latitude']
longitudes=brooklyn_data['Longitude']

radius = 500
LIMIT = 120

brooklyn_venues = getNearbyVenues(names,latitudes,longitudes,boroughs)

brooklyn_venues.shape

(2847, 8)

In [18]:
# Get nearby venues in Queens
boroughs=queens_data['Borough']
names=queens_data['Neighborhood']
latitudes=queens_data['Latitude']
longitudes=queens_data['Longitude']

radius = 500
LIMIT = 150

queens_venues = getNearbyVenues(names,latitudes,longitudes,boroughs)

queens_venues.shape

(2171, 8)

In [19]:
# Get nearby venues in Staten Island
boroughs=statenisland_data['Borough']
names=statenisland_data['Neighborhood']
latitudes=statenisland_data['Latitude']
longitudes=statenisland_data['Longitude']

radius = 500
LIMIT = 120

statenisland_venues = getNearbyVenues(names,latitudes,longitudes,boroughs)

statenisland_venues.shape

(828, 8)

In [20]:
# Concatenate venues 
ny_venues = pd.concat([bronx_venues, manhattan_venues, brooklyn_venues, queens_venues, statenisland_venues])
ny_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Bronx,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [21]:
ny_venues.shape

(10407, 8)

In [22]:
ny_venues.insert(1, 'NTA Code', '')
ny_venues.head()

,Borough,NTA Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Bronx,,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Bronx,,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Bronx,,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [23]:
!conda install shapely --yes
from shapely.geometry import Point, Polygon

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - shapely


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_0         154 KB
    sip-4.19.8                 |   py36hf484d3e_0         290 KB
    geos-3.6.2                 |       heeff764_2         1.6 MB
    matplotlib-3.1.0           |   py36h5429711_0         6.7 MB
    shapely-1.6.4              |   py36h7ef4460_0         326 KB
    openssl-1.0.2s             |       h7b6447c_0         3.1 MB
    basemap-1.2.0              |   py36h705c2d8_0        15.2 MB
    ------------------------------------------------------------
    

In [24]:
# load json file containing NTA code
with open('ny_nta_codes.geojson') as json_data:
    ny_neigh = json.load(json_data)

In [25]:
# for each Neighborhood boundaries (polygon), insert the corresponding NTA code and its' centroid coordinates

for feature in ny_neigh['features']:
    coord_list = list(feature['geometry']['coordinates'][0][0])
    coord_tuple = [tuple(item) for item in coord_list]       

    polygon = Polygon(coord_tuple)    

    for neighborhood, lat, lng in zip(ny_venues['Neighborhood'], ny_venues['Neighborhood Latitude'], ny_venues['Neighborhood Longitude']):
        point = Point(lng, lat)      
        
        if polygon.contains(point) or polygon.touches(point) or polygon.intersects(point):   
            ny_venues.loc[ny_venues['Neighborhood'] == neighborhood, ['NTA Code', 'Neighborhood Latitude','Neighborhood Longitude']] = \
                                                                     [feature['properties']['ntacode'], polygon.centroid.coords[0][0], polygon.centroid.coords[0][1]]        

In [26]:
# Checking for number of unique NTA Codes
len(ny_venues['NTA Code'].unique())

149

In [27]:
# Prepare for cleaning
ny_venues.replace(to_replace='', value='NA', inplace=True)
ny_venues[ny_venues['NTA Code'] == 'NA'].count()

Borough                   1367
NTA Code                  1367
Neighborhood              1367
Neighborhood Latitude     1367
Neighborhood Longitude    1367
Venue                     1367
Venue Latitude            1367
Venue Longitude           1367
Venue Category            1367
dtype: int64

In [28]:
#check if nta codes correspond to correct borough and make the wrong neighborhoods list
nta = {'Bronx':'BX',
       'Manhattan':'MN',
       'Brooklyn':'BK', 
       'Queens':'QN',
       'Staten Island':'SI'}

wrong = []

for borough, neighborhood, nta_code in zip(ny_venues['Borough'], ny_venues['Neighborhood'], ny_venues['NTA Code']):
    if nta[borough] != nta_code[0:2] and nta_code[0:2] != 'NA':
        #print(borough, neighborhood, nta_code)
        wrong.append([borough, neighborhood])
        
len(wrong)

159

In [29]:
# Assign 'NA' to the wrong NTA Codes
for borough, neighborhood in wrong:
    ny_venues.loc[(ny_venues['Borough'] == borough) & (ny_venues['Neighborhood'] == neighborhood), ['NTA Code']] = 'NA'
    
ny_venues[ny_venues['NTA Code'] == 'NA'].count()

Borough                   1526
NTA Code                  1526
Neighborhood              1526
Neighborhood Latitude     1526
Neighborhood Longitude    1526
Venue                     1526
Venue Latitude            1526
Venue Longitude           1526
Venue Category            1526
dtype: int64

In [30]:
# distance between 2 points
import math

def distance(p1:tuple, p2:tuple):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

In [31]:
# find the closest point to p in a set of points
def closest(p, others):
    return min(others, key = lambda i: distance(p, i))

In [32]:
# assign closest neighborhood's nta code

for borough, nta_code, neighborhood, r_lat, r_lng in zip(ny_venues['Borough'], ny_venues['NTA Code'], ny_venues['Neighborhood'], ny_venues['Neighborhood Latitude'], ny_venues['Neighborhood Longitude']):
    # find "Not Assigned" NTA Codes 
    if nta_code == 'NA':        
        # Need to find inside the corresponding Borough
        borough_data = ny_venues[ny_venues['Borough'] == borough].reset_index(drop=True)
        # neighborhood to receive the new NTA code
        r_point = (r_lat, r_lng)
        others = []
        
        # make the Other Neighborhoods list
        for n_neighborhood, n_nta_code, n_lat, n_lng in zip(borough_data['Neighborhood'], borough_data['NTA Code'], borough_data['Neighborhood Latitude'], borough_data['Neighborhood Longitude']):
            if (n_neighborhood != neighborhood) & (n_nta_code != 'NA'):
                others.append((n_lat, n_lng, n_nta_code))                
        
        # Get the closest Neighborhood                        
        cls_point = closest(r_point, others)       
        
        # Get the closest Neighborhood's latitude, longitude, and NTA Code
        ny_venues.loc[(ny_venues['Borough'] == borough) & (ny_venues['Neighborhood'] == neighborhood), ['Neighborhood Latitude','Neighborhood Longitude','NTA Code']] = cls_point        
        
# count wrong rows (NTA not corresponding to Borough)        
ny_venues[ny_venues['NTA Code'] == 'NA'].count()

Borough                   0
NTA Code                  0
Neighborhood              0
Neighborhood Latitude     0
Neighborhood Longitude    0
Venue                     0
Venue Latitude            0
Venue Longitude           0
Venue Category            0
dtype: int64

In [33]:
#check if nta codes correspond to correct borough and make the wrong neighborhoods list (again !!!)
nta = {'Bronx':'BX',
       'Manhattan':'MN',
       'Brooklyn':'BK', 
       'Queens':'QN',
       'Staten Island':'SI'}

wrong = []

for borough, neighborhood, nta_code in zip(ny_venues['Borough'], ny_venues['Neighborhood'], ny_venues['NTA Code']):
    if nta[borough] != nta_code[0:2] and nta_code[0:2] != 'NA':
        #print(borough, neighborhood, nta_code)
        wrong.append([borough, neighborhood])
        
len(wrong)

0

In [34]:
# Neighborhoods by NTA codes for grouping
#del neighborhoods['NTA Code']
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [35]:
neighborhoods.insert(1, 'NTA Code', '')
neighborhoods.head()

,Borough,NTA Code,Neighborhood,Latitude,Longitude
0,Bronx,,Wakefield,40.894705,-73.847201
1,Bronx,,Co-op City,40.874294,-73.829939
2,Bronx,,Eastchester,40.887556,-73.827806
3,Bronx,,Fieldston,40.895437,-73.905643
4,Bronx,,Riverdale,40.890834,-73.912585


In [36]:
# for each Neighborhood boundaries (polygon), insert the corresponding NTA code and its' centroid coordinates

for feature in ny_neigh['features']:
    coord_list = list(feature['geometry']['coordinates'][0][0])
    coord_tuple = [tuple(item) for item in coord_list]       

    polygon = Polygon(coord_tuple)    

    for borough, neighborhood, lat, lng in zip(neighborhoods['Borough'], neighborhoods['Neighborhood'], neighborhoods['Latitude'], neighborhoods['Longitude']):
        point = Point(lng, lat)      
        
        if polygon.contains(point) or polygon.touches(point) or polygon.intersects(point):   
            neighborhoods.loc[(neighborhoods['Borough'] == borough) & (neighborhoods['Neighborhood'] == neighborhood), ['NTA Code', 'Latitude', 'Longitude']] = \
                                                                                     [feature['properties']['ntacode'], polygon.centroid.coords[0][0], polygon.centroid.coords[0][1]] 

In [37]:
# Checking for number of unique NTA Codes
len(neighborhoods['NTA Code'].unique())

149

In [38]:
# Prepare for cleaning
neighborhoods.fillna('NA', inplace=True)
neighborhoods[neighborhoods['NTA Code'] == 'NA'].count()

Borough         0
NTA Code        0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [39]:
#check if nta codes correspond to correct borough and make the wrong neighborhoods list
nta = {'Bronx':'BX',
       'Manhattan':'MN',
       'Brooklyn':'BK', 
       'Queens':'QN',
       'Staten Island':'SI'}

wrong = []

for borough, neighborhood, nta_code in zip(neighborhoods['Borough'], neighborhoods['Neighborhood'], neighborhoods['NTA Code']):
    if nta[borough] != nta_code[0:2] and nta_code[0:2] != 'NA':
        #print(borough, neighborhood, nta_code)
        wrong.append([borough, neighborhood])
        
len(wrong)

42

In [40]:
# Assign 'NA' to the wrong NTA Codes
for borough, neighborhood in wrong:
    neighborhoods.loc[(neighborhoods['Borough'] == borough) & (neighborhoods['Neighborhood'] == neighborhood), ['NTA Code']] = 'NA'
    
neighborhoods[neighborhoods['NTA Code'] == 'NA'].count()

Borough         42
NTA Code        42
Neighborhood    42
Latitude        42
Longitude       42
dtype: int64

In [41]:
# assign closest neighborhood's nta code

for borough, nta_code, neighborhood, r_lat, r_lng in zip(neighborhoods['Borough'], neighborhoods['NTA Code'], neighborhoods['Neighborhood'], neighborhoods['Latitude'], neighborhoods['Longitude']):
    # find "Not Assigned" NTA Codes 
    if nta_code == 'NA':        
        # Need to find inside the corresponding Borough
        borough_data = neighborhoods[neighborhoods['Borough'] == borough].reset_index(drop=True)
        # neighborhood to receive the new NTA code
        r_point = (r_lat, r_lng)
        others = []
        
        # make the Other Neighborhoods list
        for n_neighborhood, n_nta_code, n_lat, n_lng in zip(borough_data['Neighborhood'], borough_data['NTA Code'], borough_data['Latitude'], borough_data['Longitude']):
            if (n_neighborhood != neighborhood) & (n_nta_code != 'NA'):
                others.append((n_lat, n_lng, n_nta_code))                
        
        # Get the closest Neighborhood                        
        cls_point = closest(r_point, others)       
        
        # Get the closest Neighborhood's latitude, longitude, and NTA Code
        neighborhoods.loc[(neighborhoods['Borough'] == borough) & (neighborhoods['Neighborhood'] == neighborhood), ['Latitude','Longitude','NTA Code']] = cls_point        
        
# count wrong rows (NTA not corresponding to Borough)        
neighborhoods[neighborhoods['NTA Code'] == 'NA'].count()

Borough         0
NTA Code        0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [42]:
#check if nta codes correspond to correct borough and make the wrong neighborhoods list (again !!!)
nta = {'Bronx':'BX',
       'Manhattan':'MN',
       'Brooklyn':'BK', 
       'Queens':'QN',
       'Staten Island':'SI'}

wrong = []

for borough, neighborhood, nta_code in zip(neighborhoods['Borough'], neighborhoods['Neighborhood'], neighborhoods['NTA Code']):
    if nta[borough] != nta_code[0:2] and nta_code[0:2] != 'NA':
        #print(borough, neighborhood, nta_code)
        wrong.append([borough, neighborhood])
        
len(wrong)

0

In [43]:
# Now let's group the names of Neighborhoods by NTA Code
nta_neigh_grouped = neighborhoods.groupby(['NTA Code','Borough','Latitude','Longitude'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()
nta_neigh_grouped.head()

,NTA Code,Borough,Latitude,Longitude,Neighborhood
0,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights
1,BK17,Brooklyn,-73.941511,40.588300,"Sheepshead Bay, Canarsie, Mill Island, Manhatt..."
2,BK19,Brooklyn,-73.961216,40.580923,Brighton Beach
3,BK21,Brooklyn,-73.991236,40.576483,"Coney Island, Sea Gate"
4,BK25,Brooklyn,-73.964334,40.599954,Homecrest


## Now, let's merge neighborhoods and venues by NTA code

In [44]:
ny_nta_venues = pd.merge(nta_neigh_grouped, ny_venues, how='left', on='NTA Code')
ny_nta_venues.shape

(10407, 13)

In [45]:
ny_nta_venues.head()

,NTA Code,Borough_x,Latitude,Longitude,Neighborhood_x,Borough_y,Neighborhood_y,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Brooklyn,Brooklyn Heights,-73.994871,40.695469,Brooklyn Historical Society,40.694942,-73.992333,History Museum
1,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Brooklyn,Brooklyn Heights,-73.994871,40.695469,Emack & Bolio's,40.694987,-73.994946,Ice Cream Shop
2,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Brooklyn,Brooklyn Heights,-73.994871,40.695469,Dellarocco's,40.694992,-73.995924,Pizza Place
3,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Brooklyn,Brooklyn Heights,-73.994871,40.695469,Sushi Gallery,40.697595,-73.993236,Sushi Restaurant
4,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Brooklyn,Brooklyn Heights,-73.994871,40.695469,Area Yoga & Spa,40.694568,-73.993514,Yoga Studio


In [46]:
ny_nta_venues.drop(['Borough_y','Neighborhood_y','Neighborhood Latitude','Neighborhood Longitude'], axis=1, inplace=True)

columns = ['NTA Code', 'Borough', 'Latitude', 'Longitude', 'Neighborhood', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
ny_nta_venues.columns = columns

ny_nta_venues.head()

,NTA Code,Borough,Latitude,Longitude,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Brooklyn Historical Society,40.694942,-73.992333,History Museum
1,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Emack & Bolio's,40.694987,-73.994946,Ice Cream Shop
2,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Dellarocco's,40.694992,-73.995924,Pizza Place
3,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Sushi Gallery,40.697595,-73.993236,Sushi Restaurant
4,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Area Yoga & Spa,40.694568,-73.993514,Yoga Studio


## Now let's use population data

In [47]:
!conda install xlrd --yes

nta_pop = pd.read_excel('Demo_NTA_Pop.xlsx')
nta_pop.head()

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    xlrd-1.2.0                 |           py36_0         188 KB

The following packages will be UPDATED:

    xlrd: 1.1.0-py37_1 --> 1.2.0-py36_0


xlrd-1.2.0           | 188 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


,GeoType,GeogName,GeoID,Borough,Tot_Pop
0,NTA2010,Bath Beach,BK27,Brooklyn,33163
1,NTA2010,Bay Ridge,BK31,Brooklyn,79134
2,NTA2010,Bedford,BK75,Brooklyn,75318
3,NTA2010,Bensonhurst East,BK29,Brooklyn,64267
4,NTA2010,Bensonhurst West,BK28,Brooklyn,91646


In [48]:
# Drop some columns
nta_pop = nta_pop.drop(['GeoType', 'GeogName', 'Borough'], axis=1)

# Rename NTA column
nta_pop.columns = ['NTA Code', 'Tot_Pop']

nta_pop.head()

,NTA Code,Tot_Pop
0,BK27,33163
1,BK31,79134
2,BK75,75318
3,BK29,64267
4,BK28,91646


In [49]:
# Sum population
Total = nta_pop['Tot_Pop'].sum()
print (Total)

8461961


In [50]:
# Merging with ny_nta_venues data
nta_neigh_pop = pd.merge(ny_nta_venues, nta_pop, how='left', on='NTA Code')
nta_neigh_pop.shape

(10407, 10)

In [51]:
nta_neigh_pop.head()

,NTA Code,Borough,Latitude,Longitude,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category,Tot_Pop
0,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Brooklyn Historical Society,40.694942,-73.992333,History Museum,24140
1,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Emack & Bolio's,40.694987,-73.994946,Ice Cream Shop,24140
2,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Dellarocco's,40.694992,-73.995924,Pizza Place,24140
3,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Sushi Gallery,40.697595,-73.993236,Sushi Restaurant,24140
4,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Area Yoga & Spa,40.694568,-73.993514,Yoga Studio,24140


In [52]:
# Analysing Total Population and number of venues
num_venues = nta_neigh_pop.groupby('NTA Code').count()[['Venue']]
num_venues.head()

,Venue
NTA Code,
BK09,100
BK17,130
BK19,44
BK21,23
BK25,45


In [53]:
num_pop = nta_neigh_pop.groupby('NTA Code').mean()[['Tot_Pop']]
num_pop.head()

,Tot_Pop
NTA Code,
BK09,24140
BK17,66293
BK19,34157
BK21,29677
BK25,44516


In [54]:
# Merge Venues e Population
venues_pop = pd.merge(num_venues, num_pop, on='NTA Code')
venues_pop.head()

,Venue,Tot_Pop
NTA Code,,
BK09,100,24140
BK17,130,66293
BK19,44,34157
BK21,23,29677
BK25,45,44516


In [55]:
# Now let's normalize the values
#venues_pop['Venue'] = venues_pop['Venue'] / venues_pop['Venue'].sum()
#venues_pop['Tot_Pop'] = venues_pop['Tot_Pop'] / venues_pop['Tot_Pop'].sum()
#venues_pop.sum()

In [56]:
# Now let's try a correlation between local population and number of venues
venues_pop.corr()

,Venue,Tot_Pop
Venue,1.000000,0.134839
Tot_Pop,0.134839,1.000000


## There seems to be no correlation at this point. Now let's try to filter only restaurants

In [57]:
nta_neigh_pop['Venue Category'].unique()

array(['History Museum', 'Ice Cream Shop', 'Pizza Place',
       'Sushi Restaurant', 'Yoga Studio', 'Deli / Bodega',
       'Japanese Restaurant', 'Gym / Fitness Center', 'Diner', 'Gym',
       'Coffee Shop', 'Asian Restaurant', 'Pet Store', 'Playground',
       'Thai Restaurant', 'Vietnamese Restaurant', 'Garden',
       'Grocery Store', 'Scenic Lookout', 'Wine Shop',
       'Mobile Phone Shop', 'Park', 'Burger Joint', 'Falafel Restaurant',
       'Farmers Market', 'Cosmetics Shop', 'Pilates Studio', 'Bakery',
       'Mexican Restaurant', 'Health & Beauty Service', 'Cycle Studio',
       'Fast Food Restaurant', 'Italian Restaurant', 'Optical Shop',
       'Flower Shop', 'Bar', 'New American Restaurant',
       'Athletics & Sports', 'American Restaurant', 'Beach', 'Café',
       'Plaza', 'Bistro', 'Indian Restaurant', 'Ramen Restaurant',
       'Food Truck', 'Middle Eastern Restaurant', 'Indie Theater',
       'Boutique', 'Juice Bar', 'Pharmacy', "Women's Store", 'Pub',
       'Burrito

In [58]:
ny_rest = nta_neigh_pop[nta_neigh_pop['Venue Category'].str.contains('Restaurant')]
ny_rest.head()

,NTA Code,Borough,Latitude,Longitude,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category,Tot_Pop
3,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Sushi Gallery,40.697595,-73.993236,Sushi Restaurant,24140
6,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Iron Chef House,40.697406,-73.992560,Japanese Restaurant,24140
11,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Saketumi Asian Bistro,40.694910,-73.994578,Asian Restaurant,24140
18,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Lantern Thai Kitchen,40.695168,-73.995621,Thai Restaurant,24140
19,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Hanco's,40.694624,-73.993457,Vietnamese Restaurant,24140


In [59]:
# Analysing Total Population and number of restaurants
num_rest = ny_rest.groupby('NTA Code').count()[['Venue']]
num_pop = ny_rest.groupby('NTA Code').mean()[['Tot_Pop']]
rest_pop = pd.merge(num_rest, num_pop, on='NTA Code')

# Now let's normalize the values
#rest_pop['Venue'] = rest_pop['Venue'] / rest_pop['Venue'].sum()
#rest_pop['Tot_Pop'] = rest_pop['Tot_Pop'] / rest_pop['Tot_Pop'].sum()

rest_pop = rest_pop.rename(columns={'Venue':'Restaurant'})

# Check correlation
rest_pop.corr()

,Restaurant,Tot_Pop
Restaurant,1.000000,0.167951
Tot_Pop,0.167951,1.000000


In [60]:
# What about Pizza Places ?
ny_pizza = nta_neigh_pop[nta_neigh_pop['Venue Category'].str.contains('Pizza Place')]

# Analysing Total Population and number of pizza places
num_pizza = ny_pizza.groupby('NTA Code').count()[['Venue']]
num_pop = ny_pizza.groupby('NTA Code').mean()[['Tot_Pop']]
pizza_pop = pd.merge(num_pizza, num_pop, on='NTA Code')

# Now let's normalize the values
#pizza_pop['Venue'] = pizza_pop['Venue'] / pizza_pop['Venue'].sum()
#pizza_pop['Tot_Pop'] = pizza_pop['Tot_Pop'] / pizza_pop['Tot_Pop'].sum()

# Check correlation
pizza_pop.corr()

,Venue,Tot_Pop
Venue,1.00000,0.01701
Tot_Pop,0.01701,1.00000


In [61]:
nta_econ = pd.read_excel('Demo_NTA_Econ.xlsx')
nta_econ.head()

,GeoID,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE
0,BK27,895,618,1223,927,1290,2074,1352,1758,706,585
1,BK31,2313,1573,3235,3047,3542,5570,4108,4827,2497,2775
2,BK75,3673,2317,3385,2509,3124,3706,2269,2847,1195,1034
3,BK29,2356,1413,2718,2185,2655,3898,2558,2416,1144,844
4,BK28,2330,1746,3383,3246,4036,5117,3428,3743,1649,1231


In [62]:
# Merging with number of venues
nta_econ = nta_econ.rename(columns={'GeoID': 'NTA Code'})

nta_neigh_econ_venues = pd.merge(nta_econ, num_venues, how='left', on='NTA Code')
nta_neigh_econ_venues.head()

,NTA Code,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE,Venue
0,BK27,895,618,1223,927,1290,2074,1352,1758,706,585,50.0
1,BK31,2313,1573,3235,3047,3542,5570,4108,4827,2497,2775,156.0
2,BK75,3673,2317,3385,2509,3124,3706,2269,2847,1195,1034,NaN
3,BK29,2356,1413,2718,2185,2655,3898,2558,2416,1144,844,NaN
4,BK28,2330,1746,3383,3246,4036,5117,3428,3743,1649,1231,36.0


In [63]:
nta_neigh_econ_venues = nta_neigh_econ_venues.fillna(0)
nta_neigh_econ_venues.head()

,NTA Code,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE,Venue
0,BK27,895,618,1223,927,1290,2074,1352,1758,706,585,50.0
1,BK31,2313,1573,3235,3047,3542,5570,4108,4827,2497,2775,156.0
2,BK75,3673,2317,3385,2509,3124,3706,2269,2847,1195,1034,0.0
3,BK29,2356,1413,2718,2185,2655,3898,2558,2416,1144,844,0.0
4,BK28,2330,1746,3383,3246,4036,5117,3428,3743,1649,1231,36.0


In [64]:
nta_neigh_econ_venues.corr()

,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE,Venue
HHIU10E,1.000000,0.918252,0.852092,0.779253,0.730053,0.588271,0.387581,0.243045,0.124522,0.118168,0.114436
HHI10t14E,0.918252,1.000000,0.905218,0.829617,0.772210,0.629357,0.416760,0.264286,0.118915,0.077813,0.112048
HHI15t24E,0.852092,0.905218,1.000000,0.943470,0.901070,0.796670,0.594155,0.393992,0.208036,0.116915,0.112839
HHI25t34E,0.779253,0.829617,0.943470,1.000000,0.945250,0.864680,0.669828,0.449590,0.248828,0.117626,0.081814
HHI35t49E,0.730053,0.772210,0.901070,0.945250,1.000000,0.919583,0.769699,0.565295,0.353466,0.194130,0.093189
HHI50t74E,0.588271,0.629357,0.796670,0.864680,0.919583,1.000000,0.917182,0.775378,0.588475,0.394129,0.143122
HHI75t99E,0.387581,0.416760,0.594155,0.669828,0.769699,0.917182,1.000000,0.917646,0.774792,0.567102,0.192746
HI100t149E,0.243045,0.264286,0.393992,0.449590,0.565295,0.775378,0.917646,1.000000,0.931182,0.766459,0.230243
HI150t199E,0.124522,0.118915,0.208036,0.248828,0.353466,0.588475,0.774792,0.931182,1.000000,0.901902,0.265289
HHI200plE,0.118168,0.077813,0.116915,0.117626,0.194130,0.394129,0.567102,0.766459,0.901902,1.000000,0.321676


In [65]:
# Now let's add the number of Restaurants and population
nta_neigh_econ_rest = pd.merge(nta_neigh_econ_venues, rest_pop, how='left', on='NTA Code')
nta_neigh_econ_rest.fillna(0, inplace=True)
nta_neigh_econ_rest.head()

,NTA Code,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE,Venue,Restaurant,Tot_Pop
0,BK27,895,618,1223,927,1290,2074,1352,1758,706,585,50.0,16.0,33163.0
1,BK31,2313,1573,3235,3047,3542,5570,4108,4827,2497,2775,156.0,43.0,79134.0
2,BK75,3673,2317,3385,2509,3124,3706,2269,2847,1195,1034,0.0,0.0,0.0
3,BK29,2356,1413,2718,2185,2655,3898,2558,2416,1144,844,0.0,0.0,0.0
4,BK28,2330,1746,3383,3246,4036,5117,3428,3743,1649,1231,36.0,18.0,91646.0


In [66]:
nta_neigh_econ_rest.corr()

,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE,Venue,Restaurant,Tot_Pop
HHIU10E,1.000000,0.918252,0.852092,0.779253,0.730053,0.588271,0.387581,0.243045,0.124522,0.118168,0.114436,0.127685,0.424587
HHI10t14E,0.918252,1.000000,0.905218,0.829617,0.772210,0.629357,0.416760,0.264286,0.118915,0.077813,0.112048,0.131462,0.426118
HHI15t24E,0.852092,0.905218,1.000000,0.943470,0.901070,0.796670,0.594155,0.393992,0.208036,0.116915,0.112839,0.149204,0.570009
HHI25t34E,0.779253,0.829617,0.943470,1.000000,0.945250,0.864680,0.669828,0.449590,0.248828,0.117626,0.081814,0.128854,0.657357
HHI35t49E,0.730053,0.772210,0.901070,0.945250,1.000000,0.919583,0.769699,0.565295,0.353466,0.194130,0.093189,0.136783,0.669486
HHI50t74E,0.588271,0.629357,0.796670,0.864680,0.919583,1.000000,0.917182,0.775378,0.588475,0.394129,0.143122,0.191314,0.685637
HHI75t99E,0.387581,0.416760,0.594155,0.669828,0.769699,0.917182,1.000000,0.917646,0.774792,0.567102,0.192746,0.238779,0.623103
HI100t149E,0.243045,0.264286,0.393992,0.449590,0.565295,0.775378,0.917646,1.000000,0.931182,0.766459,0.230243,0.259324,0.501306
HI150t199E,0.124522,0.118915,0.208036,0.248828,0.353466,0.588475,0.774792,0.931182,1.000000,0.901902,0.265289,0.285509,0.390748
HHI200plE,0.118168,0.077813,0.116915,0.117626,0.194130,0.394129,0.567102,0.766459,0.901902,1.000000,0.321676,0.322536,0.279634


## There's a strong correlation between total number of venues and the number of restaurants
## Let's analyse the correlation between the number of other venues and the number of restaurants

In [67]:
other_venues = nta_neigh_pop[~nta_neigh_pop['Venue Category'].str.contains('Restaurant|Pizza')]

num_other_venues = other_venues.groupby('NTA Code').count()[['Venue']]
num_other_venues.head()

,Venue
NTA Code,
BK09,75
BK17,99
BK19,29
BK21,21
BK25,32


In [68]:
# Now Let's Replace the venue column
del nta_neigh_econ_rest['Venue']

nta_econ_other_venues = pd.merge(nta_neigh_econ_rest, num_other_venues, how='left', on='NTA Code')
nta_econ_other_venues.fillna(0, inplace=True)

nta_econ_other_venues.head()

,NTA Code,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE,Restaurant,Tot_Pop,Venue
0,BK27,895,618,1223,927,1290,2074,1352,1758,706,585,16.0,33163.0,32.0
1,BK31,2313,1573,3235,3047,3542,5570,4108,4827,2497,2775,43.0,79134.0,108.0
2,BK75,3673,2317,3385,2509,3124,3706,2269,2847,1195,1034,0.0,0.0,0.0
3,BK29,2356,1413,2718,2185,2655,3898,2558,2416,1144,844,0.0,0.0,0.0
4,BK28,2330,1746,3383,3246,4036,5117,3428,3743,1649,1231,18.0,91646.0,17.0


In [69]:
# Now, let's see if there's a high correlation between number of other venues and number of restaurants
nta_econ_other_venues.corr()

,HHIU10E,HHI10t14E,HHI15t24E,HHI25t34E,HHI35t49E,HHI50t74E,HHI75t99E,HI100t149E,HI150t199E,HHI200plE,Restaurant,Tot_Pop,Venue
HHIU10E,1.000000,0.918252,0.852092,0.779253,0.730053,0.588271,0.387581,0.243045,0.124522,0.118168,0.127685,0.424587,0.107652
HHI10t14E,0.918252,1.000000,0.905218,0.829617,0.772210,0.629357,0.416760,0.264286,0.118915,0.077813,0.131462,0.426118,0.102616
HHI15t24E,0.852092,0.905218,1.000000,0.943470,0.901070,0.796670,0.594155,0.393992,0.208036,0.116915,0.149204,0.570009,0.096955
HHI25t34E,0.779253,0.829617,0.943470,1.000000,0.945250,0.864680,0.669828,0.449590,0.248828,0.117626,0.128854,0.657357,0.061246
HHI35t49E,0.730053,0.772210,0.901070,0.945250,1.000000,0.919583,0.769699,0.565295,0.353466,0.194130,0.136783,0.669486,0.075294
HHI50t74E,0.588271,0.629357,0.796670,0.864680,0.919583,1.000000,0.917182,0.775378,0.588475,0.394129,0.191314,0.685637,0.123828
HHI75t99E,0.387581,0.416760,0.594155,0.669828,0.769699,0.917182,1.000000,0.917646,0.774792,0.567102,0.238779,0.623103,0.174549
HI100t149E,0.243045,0.264286,0.393992,0.449590,0.565295,0.775378,0.917646,1.000000,0.931182,0.766459,0.259324,0.501306,0.220706
HI150t199E,0.124522,0.118915,0.208036,0.248828,0.353466,0.588475,0.774792,0.931182,1.000000,0.901902,0.285509,0.390748,0.260283
HHI200plE,0.118168,0.077813,0.116915,0.117626,0.194130,0.394129,0.567102,0.766459,0.901902,1.000000,0.322536,0.279634,0.325754


In [70]:
from scipy import stats
# To avoid errors calculating the pearson coefficient, let's drop the 0 rows
nta_econ_other_venues = nta_econ_other_venues[nta_econ_other_venues['Restaurant'] != 0]
# Calculating pearson coefficient and p_value
pearson_coef, p_value = stats.pearsonr(nta_econ_other_venues['Restaurant'], nta_econ_other_venues['Venue'])
print("The Pearson Correlation Coefficient is", pearson_coef, " with a P-value of P =", p_value)

The Pearson Correlation Coefficient is 0.955318260458393  with a P-value of P = 1.6260089240426387e-76


## So, the restaurants tend to be near other venues !!!

In [71]:
# Now let's rank the top 10 neighborhoods by number of venues
nta_econ_other_venues_sorted = nta_econ_other_venues.sort_values(by='Venue', axis=0, ascending=False)

# Let's drop the economic columns
cols = [col for col in nta_econ_other_venues_sorted.columns if col.lower()[:1] != 'h']

nta_econ_other_venues_sorted = nta_econ_other_venues_sorted[cols]

nta_econ_other_venues_sorted.head(10)

,NTA Code,Restaurant,Tot_Pop,Venue
54,MN27,264.0,44898.0,652.0
71,MN24,132.0,42979.0,364.0
61,MN13,98.0,77205.0,275.0
17,BK38,54.0,42245.0,242.0
96,QN15,49.0,54264.0,183.0
12,BK33,59.0,42463.0,182.0
67,MN17,48.0,27811.0,150.0
192,BX59,61.0,28437.0,146.0
74,MN40,48.0,58593.0,144.0
34,BK73,59.0,54330.0,133.0


In [72]:
# Now, let's see the top 10 neighborhoods locations
top_10_neigh = nta_econ_other_venues_sorted.head(10)

top_10_neigh = pd.merge(top_10_neigh, nta_neigh_grouped, how='left', on='NTA Code')
top_10_neigh

,NTA Code,Restaurant,Tot_Pop,Venue,Borough,Latitude,Longitude,Neighborhood
0,MN27,264.0,44898.0,652.0,Manhattan,-73.994630,40.715607,"Marble Hill, Chinatown, Inwood, Yorkville, Len..."
1,MN24,132.0,42979.0,364.0,Manhattan,-74.004747,40.720766,"Greenwich Village, Tribeca, Little Italy, Soho..."
2,MN13,98.0,77205.0,275.0,Manhattan,-73.999021,40.747585,"Clinton, Chelsea, Flatiron, Hudson Yards"
3,BK38,54.0,42245.0,242.0,Brooklyn,-73.985987,40.692927,"Downtown, Boerum Hill, Vinegar Hill, Dumbo"
4,QN15,49.0,54264.0,183.0,Queens,-73.755317,40.602455,"Howard Beach, College Point, Far Rockaway, Bro..."
5,BK33,59.0,42463.0,182.0,Brooklyn,-74.003899,40.678772,"Cobble Hill, Carroll Gardens, Red Hook"
6,MN17,48.0,27811.0,150.0,Manhattan,-73.983503,40.755734,"Midtown, Midtown South"
7,BX59,61.0,28437.0,146.0,Bronx,-73.848643,40.832100,"City Island, Mott Haven, Port Morris, Soundvie..."
8,MN40,48.0,58593.0,144.0,Manhattan,-73.961174,40.774730,"Upper East Side, Carnegie Hill"
9,BK73,59.0,54330.0,133.0,Brooklyn,-73.958625,40.714916,"North Side, South Side"


In [73]:
# Now let's visualize those neighborhoods on a map
# Defining location
address = 'New York, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Top 10 neighborhoods using latitude and longitude values
map_top_10 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, venue in zip(top_10_neigh['Latitude'], top_10_neigh['Longitude'], top_10_neigh['Borough'], top_10_neigh['Neighborhood'], top_10_neigh['Venue']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=int(venue)//10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_top_10)  
    
map_top_10

In [74]:
# Now let's see if there's a correlation with ethnical data
# Let's take only the restaurants
# one hot encoding
ny_rest_cat = pd.get_dummies(ny_rest[['Venue Category']], prefix="", prefix_sep="")

# add NTA Code column back to dataframe
ny_rest_cat['NTA Code'] = ny_rest['NTA Code'] 

# move neighborhood column to the first column
fixed_columns = [ny_rest_cat.columns[-1]] + list(ny_rest_cat.columns[:-1])
ny_rest_cat = ny_rest_cat[fixed_columns]

ny_rest_cat.head()

,NTA Code,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,...,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
3,BK09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,BK09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,BK09,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,BK09,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
19,BK09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


#### Next, let's group rows by NTA Code and by taking the mean of the frequency of occurrence of each category

In [75]:
ny_rest_cat_grouped = ny_rest_cat.groupby('NTA Code').mean().reset_index()
ny_rest_cat_grouped.head()

,NTA Code,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,...,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,BK09,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,0.0,...,0.0,0.0,0.0,0.090909,0.0,0.00,0.0,0.0,0.0,0.045455
1,BK17,0.0,0.0,0.240000,0.0,0.0,0.080000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.040000,0.0,0.12,0.0,0.0,0.0,0.000000
2,BK19,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000
3,BK21,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000
4,BK25,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.090909


In [76]:
# Now let's add ethnical data
nta_ethn = pd.read_excel('Demo_NTA_Ethn.xlsx')

nta_ethn.rename(columns={'GeoID':'NTA Code'}, inplace=True)

# Calculating percentage for each column
nta_ethn['Pop_Hsp'] = nta_ethn['Pop_Hsp']/nta_ethn['Pop_Hsp'].sum()
nta_ethn['Pop_Wht'] = nta_ethn['Pop_Wht']/nta_ethn['Pop_Wht'].sum()
nta_ethn['Pop_Blk'] = nta_ethn['Pop_Blk']/nta_ethn['Pop_Blk'].sum()
nta_ethn['Pop_Ind'] = nta_ethn['Pop_Ind']/nta_ethn['Pop_Ind'].sum()
nta_ethn['Pop_Asn'] = nta_ethn['Pop_Asn']/nta_ethn['Pop_Asn'].sum()
nta_ethn['Pop_Pac'] = nta_ethn['Pop_Pac']/nta_ethn['Pop_Pac'].sum()
nta_ethn['Pop_Oth'] = nta_ethn['Pop_Oth']/nta_ethn['Pop_Oth'].sum()
nta_ethn['Pop_2Rc'] = nta_ethn['Pop_2Rc']/nta_ethn['Pop_2Rc'].sum()

nta_ethn.head()

,NTA Code,Pop_Hsp,Pop_Wht,Pop_Blk,Pop_Ind,Pop_Asn,Pop_Pac,Pop_Oth,Pop_2Rc
0,BK27,0.002385,0.005308,0.000215,0.000747,0.010408,0.009660,0.000943,0.002130
1,BK31,0.005750,0.018387,0.000804,0.012291,0.010009,0.006329,0.002006,0.009433
2,BK75,0.005973,0.009673,0.016264,0.004482,0.001990,0.000000,0.001980,0.007660
3,BK29,0.003223,0.010950,0.000283,0.000000,0.021606,0.000000,0.004624,0.004716
4,BK28,0.005443,0.014327,0.000426,0.007538,0.032274,0.000000,0.004239,0.005246


In [77]:
# Now let's merge restaurant categories with Ethnical data
nta_rest_ethn = pd.merge(ny_rest_cat_grouped, nta_ethn, how='left', on='NTA Code')
nta_rest_ethn.head()

,NTA Code,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Brazilian Restaurant,...,Venezuelan Restaurant,Vietnamese Restaurant,Pop_Hsp,Pop_Wht,Pop_Blk,Pop_Ind,Pop_Asn,Pop_Pac,Pop_Oth,Pop_2Rc
0,BK09,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,0.0,...,0.0,0.045455,0.000793,0.006396,0.000637,0.000000,0.002023,0.003331,0.000824,0.007501
1,BK17,0.0,0.0,0.240000,0.0,0.0,0.080000,0.0,0.0,0.0,...,0.0,0.000000,0.002363,0.015914,0.002523,0.011340,0.009800,0.026649,0.000651,0.004809
2,BK19,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.001588,0.008698,0.000139,0.010118,0.004437,0.000000,0.001887,0.005629
3,BK21,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.003164,0.003376,0.004704,0.006180,0.002719,0.000000,0.003229,0.002527
4,BK25,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,0.0,...,0.0,0.090909,0.001812,0.012701,0.000227,0.007334,0.003351,0.052965,0.001090,0.005014


In [78]:
# Finally, let's try correlation
c = nta_rest_ethn.corr().abs()

s = c.unstack()
so = s.sort_values(kind="quicksort", ascending=False)

# There wasn´t any correlation with ethnical data before line 175
print(so[175:185])

Malay Restaurant          Pop_Asn                     0.569465
Mediterranean Restaurant  French Restaurant           0.564181
French Restaurant         Mediterranean Restaurant    0.564181
Polish Restaurant         Egyptian Restaurant         0.562385
Egyptian Restaurant       Polish Restaurant           0.562385
Hawaiian Restaurant       Jewish Restaurant           0.541181
Jewish Restaurant         Hawaiian Restaurant         0.541181
Pop_2Rc                   Pop_Ind                     0.530423
Pop_Ind                   Pop_2Rc                     0.530423
Argentinian Restaurant    Indonesian Restaurant       0.521288
dtype: float64


In [79]:
print(so[205:215])

Russian Restaurant               Eastern European Restaurant        0.484769
Molecular Gastronomy Restaurant  Udon Restaurant                    0.474075
Udon Restaurant                  Molecular Gastronomy Restaurant    0.474075
                                 Modern European Restaurant         0.474075
Modern European Restaurant       Udon Restaurant                    0.474075
Fast Food Restaurant             Pop_Oth                            0.456505
Pop_Oth                          Fast Food Restaurant               0.456505
Pop_Blk                          Caribbean Restaurant               0.446637
Caribbean Restaurant             Pop_Blk                            0.446637
Cantonese Restaurant             Hotpot Restaurant                  0.437259
dtype: float64


In [80]:
print(so[225:245])

Japanese Restaurant        Greek Restaurant             0.417007
Cuban Restaurant           Hawaiian Restaurant          0.410228
Hawaiian Restaurant        Cuban Restaurant             0.410228
Comfort Food Restaurant    Sushi Restaurant             0.404042
Sushi Restaurant           Comfort Food Restaurant      0.404042
Scandinavian Restaurant    Argentinian Restaurant       0.401107
Argentinian Restaurant     Swiss Restaurant             0.401107
Venezuelan Restaurant      Argentinian Restaurant       0.401107
Argentinian Restaurant     Scandinavian Restaurant      0.401107
Swiss Restaurant           Argentinian Restaurant       0.401107
Argentinian Restaurant     Venezuelan Restaurant        0.401107
Kebab Restaurant           Hawaiian Restaurant          0.400630
Hawaiian Restaurant        Kebab Restaurant             0.400630
Argentinian Restaurant     Udon Restaurant              0.398896
Udon Restaurant            Argentinian Restaurant       0.398896
Pop_Asn                  

In [81]:
print(so[245:265])

Szechuan Restaurant        Cantonese Restaurant         0.390160
Peruvian Restaurant        Colombian Restaurant         0.383935
Colombian Restaurant       Peruvian Restaurant          0.383935
Szechuan Restaurant        Hawaiian Restaurant          0.380399
Hawaiian Restaurant        Szechuan Restaurant          0.380399
Spanish Restaurant         Pop_Hsp                      0.378745
Pop_Hsp                    Spanish Restaurant           0.378745
Ramen Restaurant           Japanese Curry Restaurant    0.376244
Japanese Curry Restaurant  Ramen Restaurant             0.376244
Indonesian Restaurant      Thai Restaurant              0.373056
Thai Restaurant            Indonesian Restaurant        0.373056
Pop_Asn                    Colombian Restaurant         0.372384
Colombian Restaurant       Pop_Asn                      0.372384
Pop_Wht                    Mediterranean Restaurant     0.370687
Mediterranean Restaurant   Pop_Wht                      0.370687
Romanian Restaurant      

In [82]:
print(so[265:285])

Tex-Mex Restaurant               Pop_Pac                            0.360964
Dumpling Restaurant              Hotpot Restaurant                  0.357557
Hotpot Restaurant                Dumpling Restaurant                0.357557
Colombian Restaurant             Thai Restaurant                    0.357219
Thai Restaurant                  Colombian Restaurant               0.357219
Malay Restaurant                 Cantonese Restaurant               0.355304
Cantonese Restaurant             Malay Restaurant                   0.355304
Pop_Asn                          Pop_2Rc                            0.351463
Pop_2Rc                          Pop_Asn                            0.351463
Arepa Restaurant                 Thai Restaurant                    0.347579
Thai Restaurant                  Arepa Restaurant                   0.347579
Pop_Wht                          Japanese Restaurant                0.347335
Japanese Restaurant              Pop_Wht                            0.347335

In [83]:
ny_rest.head()

,NTA Code,Borough,Latitude,Longitude,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category,Tot_Pop
3,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Sushi Gallery,40.697595,-73.993236,Sushi Restaurant,24140
6,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Iron Chef House,40.697406,-73.992560,Japanese Restaurant,24140
11,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Saketumi Asian Bistro,40.694910,-73.994578,Asian Restaurant,24140
18,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Lantern Thai Kitchen,40.695168,-73.995621,Thai Restaurant,24140
19,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Hanco's,40.694624,-73.993457,Vietnamese Restaurant,24140


In [84]:
ny_rest_rank = ny_rest['Venue Category'].value_counts().reset_index()
ny_rest_rank.columns = ['Venue Category', 'Count']
ny_rest_rank.head(10)

,Venue Category,Count
0,Italian Restaurant,310
1,Chinese Restaurant,230
2,Mexican Restaurant,174
3,American Restaurant,170
4,Sushi Restaurant,116
5,Japanese Restaurant,98
6,Fast Food Restaurant,98
7,Thai Restaurant,89
8,Restaurant,85
9,Seafood Restaurant,82


In [85]:
ny_rest_count = ny_rest.groupby(['Venue Category', 'NTA Code']).size().unstack(fill_value=0)
ny_rest_count.head()

NTA Code,BK09,BK17,BK19,BK21,BK25,BK26,BK27,BK28,BK31,BK33,...,SI14,SI22,SI24,SI25,SI32,SI35,SI36,SI37,SI45,SI54
Venue Category,,,,,,,,,,,,,,,,,,,,,
Afghan Restaurant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
African Restaurant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
American Restaurant,2,6,0,0,1,0,0,0,4,4,...,2,2,1,0,0,1,1,1,0,0
Arepa Restaurant,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Argentinian Restaurant,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [86]:
# Top ten restaurant category
ny_top_ten_rest = ny_rest.loc[ny_rest['Venue Category'].isin(['Italian Restaurant','Chinese Restaurant', \
                                                                            'Mexican Restaurant','American Restaurant', \
                                                                            'Sushi Restaurant','Fast Food Restaurant', \
                                                                            'Japanese Restaurant','Restaurant', \
                                                                            'Sea Food', 'Thai Restaurant'])]
ny_top_ten_rest.head()

,NTA Code,Borough,Latitude,Longitude,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category,Tot_Pop
3,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Sushi Gallery,40.697595,-73.993236,Sushi Restaurant,24140
6,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Iron Chef House,40.697406,-73.992560,Japanese Restaurant,24140
18,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Lantern Thai Kitchen,40.695168,-73.995621,Thai Restaurant,24140
35,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,Chipotle Mexican Grill,40.694349,-73.991831,Mexican Restaurant,24140
39,BK09,Brooklyn,-73.994871,40.695469,Brooklyn Heights,B.GOOD,40.694697,-73.993800,Fast Food Restaurant,24140


In [87]:
categories = {'Italian Restaurant':'red', \
            'Chinese Restaurant':'blue', \
            'Mexican Restaurant':'gray', \
            'American Restaurant':'yellow', \
            'Sushi Restaurant':'purple', \
            'Fast Food Restaurant':'brown', \
            'Japanese Restaurant':'black', \
            'Restaurant':'green', \
            'Sea Food Restaurant':'orange', \
            'Thai Restaurant':'darkblue'}

# create map of restaurant categories using latitude and longitude values
map_ny_rest_cat = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, cat in zip(ny_top_ten_rest['Venue Latitude'], ny_top_ten_rest['Venue Longitude'], ny_top_ten_rest['Venue Category']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,        
        color=categories[cat],
        fill=True,
        fill_color=categories[cat],
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny_rest_cat)  
    
map_ny_rest_cat

In [88]:
# one hot encoding
ny_rest_10_cat = pd.get_dummies(ny_top_ten_rest[['Venue Category']], prefix="", prefix_sep="")

# add NTA Code column back to dataframe
ny_rest_10_cat['NTA Code'] = ny_top_ten_rest['NTA Code'] 

# move neighborhood column to the first column
fixed_columns = [ny_rest_10_cat.columns[-1]] + list(ny_rest_10_cat.columns[:-1])
ny_rest_10_cat = ny_rest_10_cat[fixed_columns]

ny_rest_10_cat.head()

,NTA Code,American Restaurant,Chinese Restaurant,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Restaurant,Sushi Restaurant,Thai Restaurant
3,BK09,0,0,0,0,0,0,0,1,0
6,BK09,0,0,0,0,1,0,0,0,0
18,BK09,0,0,0,0,0,0,0,0,1
35,BK09,0,0,0,0,0,1,0,0,0
39,BK09,0,0,1,0,0,0,0,0,0


In [89]:
top_10_grouped = ny_rest_10_cat.groupby('NTA Code').mean().reset_index()
top_10_grouped.head()

,NTA Code,American Restaurant,Chinese Restaurant,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Restaurant,Sushi Restaurant,Thai Restaurant
0,BK09,0.166667,0.083333,0.083333,0.250000,0.083333,0.083333,0.000000,0.083333,0.166667
1,BK17,0.428571,0.142857,0.000000,0.142857,0.000000,0.071429,0.142857,0.000000,0.071429
2,BK19,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.428571,0.285714,0.000000
3,BK25,0.125000,0.250000,0.125000,0.000000,0.000000,0.250000,0.000000,0.250000,0.000000
4,BK26,0.000000,0.750000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000


In [90]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [91]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['NTA Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
top_10_sorted = pd.DataFrame(columns=columns)
top_10_sorted['NTA Code'] = top_10_grouped['NTA Code']

for ind in np.arange(top_10_grouped.shape[0]):
    top_10_sorted.iloc[ind, 1:] = return_most_common_venues(top_10_grouped.iloc[ind, :], num_top_venues)

top_10_sorted.head()

,NTA Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,BK09,Italian Restaurant,Thai Restaurant,American Restaurant,Sushi Restaurant,Mexican Restaurant
1,BK17,American Restaurant,Restaurant,Italian Restaurant,Chinese Restaurant,Thai Restaurant
2,BK19,Restaurant,Sushi Restaurant,Fast Food Restaurant,Thai Restaurant,Mexican Restaurant
3,BK25,Sushi Restaurant,Mexican Restaurant,Chinese Restaurant,Fast Food Restaurant,American Restaurant
4,BK26,Chinese Restaurant,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Restaurant


In [92]:
hottest_neigh = nta_econ_other_venues_sorted['NTA Code'].head(10)
hottest_neigh

54     MN27
71     MN24
61     MN13
17     BK38
96     QN15
12     BK33
67     MN17
192    BX59
74     MN40
34     BK73
Name: NTA Code, dtype: object

In [93]:
# Now let's see the 5 ost common Restaurant Categories for the 10 hottest places in NYC
list_of_nta = ['MN27','MN24','MN13','BK38','QN15','BK33','MN17','BX59','MN40','BK73']

hottest_cat = top_10_sorted[top_10_sorted['NTA Code'].isin(list_of_nta)]
hottest_cat

,NTA Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,BK33,Italian Restaurant,Thai Restaurant,American Restaurant,Sushi Restaurant,Restaurant
12,BK38,American Restaurant,Thai Restaurant,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
25,BK73,American Restaurant,Chinese Restaurant,Japanese Restaurant,Sushi Restaurant,Mexican Restaurant
54,BX59,Italian Restaurant,Chinese Restaurant,Fast Food Restaurant,American Restaurant,Mexican Restaurant
62,MN13,Italian Restaurant,American Restaurant,Japanese Restaurant,Thai Restaurant,Restaurant
64,MN17,Japanese Restaurant,American Restaurant,Italian Restaurant,Sushi Restaurant,Restaurant
68,MN24,Italian Restaurant,American Restaurant,Sushi Restaurant,Chinese Restaurant,Japanese Restaurant
69,MN27,Italian Restaurant,Mexican Restaurant,Sushi Restaurant,Chinese Restaurant,American Restaurant
73,MN40,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,American Restaurant,Mexican Restaurant
81,QN15,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Mexican Restaurant,Japanese Restaurant
